# “Hello, Sionna!"

The following notebook implements the “Hello, Sionna!” example from the [Sionna whitepaper](https://arxiv.org/pdf/2203.11854.pdf) and the [NVIDIA blog post](https://developer.nvidia.com/blog/jumpstarting-link-level-simulations-with-sionna/). 
The transmission of a batch of LDPC codewords over an AWGN channel using 16QAM modulation is simulated. This example shows how Sionna layers are instantiated and applied to a previously defined tensor. The coding style follows the [functional API](https://www.tensorflow.org/guide/keras/functional) of Keras.

The [official documentation](https://nvlabs.github.io/sionna) provides key material on how to use Sionna and how its components are implemented.
Many more [tutorials](https://nvlabs.github.io/sionna/tutorials.html) are available online.

In [1]:
import os
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna
    
# IPython "magic function" for inline plots
%matplotlib inline
import matplotlib.pyplot as plt

# Import required Sionna components
from sionna.mapping import Constellation, Mapper, Demapper
from sionna.utils import BinarySource, compute_ber, BinaryCrossentropy
from sionna.channel import AWGN
from sionna.fec.ldpc import LDPC5GEncoder, LDPC5GDecoder

# For the implementation of the neural receiver
import tensorflow as tf
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.layers import Dense, Layer

Let us define the required transmitter and receiver components for the transmission.

In [2]:
batch_size = 1024
n = 1000 # codeword length
k = 500 # information bits per codeword
m = 4 # number of bits per symbol
snr = 10

c = Constellation("qam", m)
b = BinarySource()([batch_size, k])
u = LDPC5GEncoder(k, n)(b)
x = Mapper(constellation=c)(u)
y = AWGN()([x, 1/snr])
llr = Demapper("app", constellation=c)([y, 1/snr])
b_hat = LDPC5GDecoder(LDPC5GEncoder(k, n))(llr)

We can now directly calculate the simulated bit-error-rate (BER) for the whole batch of 1024 codewords.

In [3]:
ber = compute_ber(b, b_hat)
print(f"Coded BER = {ber :.3f}")

Coded BER = 0.000


## Automatic Gradient Computation

One of the key advantages of Sionna is that components can be made trainable or replaced by neural networks. In the following example, we have made the [Constellation](https://nvlabs.github.io/sionna/api/mapping.html#constellation) trainable and replaced [Demapper](https://nvlabs.github.io/sionna/api/mapping.html#demapping) with a NeuralDemapper, which is just a neural network defined through [Keras](https://keras.io).

In [4]:
# Let us define the Neural Demapper
class NeuralDemapper(Layer):
    def build(self, input_shape):
        # Initialize the neural network layers
        self._dense1 = Dense(16, activation="relu")
        self._dense2 = Dense(m)

    def call(self, inputs):
        y, no = inputs

        # Stack noise variance, real and imaginary
        # parts of each symbol. The input to the
        # neural network is [Re(y_i), Im(y_i), no].
        no = no*tf.ones(tf.shape(y))
        llr = tf.stack([tf.math.real(y),
                        tf.math.imag(y),
                        no], axis=-1)

        # Compute neural network output
        llr = self._dense1(llr)
        llr = self._dense2(llr)

        # Reshape to [batch_size, n]
        llr = tf.reshape(llr, [batch_size, -1])
        return llr

Now, we can simply replace the *classical* demapper with the previously defined NeuralDemapper and set the Constellation object trainable.

In [5]:
with tf.GradientTape() as tape: # Watch gradients
    c = Constellation("qam", m, trainable=True) # Constellation object is now trainable
    b = BinarySource()([batch_size, k])
    u = LDPC5GEncoder(k, n)(b)
    x = Mapper(constellation=c)(u)
    y = AWGN()([x, 1/snr])
    llr = NeuralDemapper()([y, 1/snr]) # Replaced by the NeuralDemapper
    loss = BinaryCrossentropy(from_logits=True)(u, llr)

# Use TensorFlows automatic gradient computation
grad = tape.gradient(loss, tape.watched_variables())

print("Gradients of the Constellation object: ", grad[0])

Gradients of the Constellation:  tf.Tensor(
[[-0.00329827 -0.00322376 -0.00074452  0.00339787 -0.00513376 -0.00157427
   0.00207564  0.00579527 -0.00197416 -0.01357635  0.00363045 -0.00814517
   0.00536535  0.00455021  0.0116162   0.00674398]
 [-0.00412695  0.00536264  0.00231561  0.00733877 -0.00158195  0.00073293
   0.00541865  0.00879211 -0.0044257   0.00550126 -0.00092697  0.00692451
  -0.00466044 -0.00021787 -0.00036102  0.00797399]], shape=(2, 16), dtype=float32)


We could now use these gradients to write a custom training loop to jointly update the Constellation and the NeuralDemapper. For further details, we refer to the [Sionna Tutorial Part 2](https://nvlabs.github.io/sionna/examples/Sionna_tutorial_part2.html) Jupyter notebook.